# 導入函式 & 執行

In [6]:
%run using_func.ipynb

# 準備
---
1. 連線登入
2. 查看TABLES狀態

In [7]:
conn = conn_login()
cursor = conn.cursor()

In [8]:
cursor.execute("SHOW TABLES")
Tables = cursor.fetchall()
for i in Tables:
    print(i[0])

customers
drinks
orders


# TALBES遍歷

In [9]:
DF_tables = {}
for table in Tables:
    tablename = table[0]
    DF_tables = SQL_to_DF(tablename, DF_tables)
cursor.close()
conn.close()

In [10]:
DF_display(DF_tables)

customers


,id,name,phone,birthday
0,1,Johny,0980123123,1997-04-22
1,2,Wendy,0919456456,1999-07-07


drinks


,id,name,price,cost
0,1,阿華田,65,20
1,2,百香紅茶,45,10
2,3,四季春茶,25,5
3,4,愛玉冰茶,50,7
4,5,冰咖啡,90,30
5,6,好喝檸檬水,50,20


orders


,id,drinks_id,customers_id,qty,order_time
0,1,1,2,4,2019-07-14 13:55:56
1,2,2,2,3,2019-07-14 14:21:23
2,3,3,2,9,2019-07-14 14:26:12
3,4,3,1,2,2019-07-14 18:51:50


# 問題1
---
1. 飲料店每位顧客的平均銷售額是多少？
2. 誰是購買額最高的兩名顧客？
3. 這個月生日的顧客有誰？

In [11]:
# 1.飲料店每位顧客的平均銷售額是多少？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
    SELECT c.`name`, (
        SELECT AVG(d.`price` * o.`qty`)
        FROM `orders` o
        JOIN `drinks` d ON o.`drinks_id` = d.`id`
        WHERE o.`customers_id` = c.`id`
    ) AS `AVG`
FROM `customers` AS c;
''')

res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,name,AVG
0,Johny,50.0000
1,Wendy,206.6667


In [12]:
# 2.誰是購買額最高的兩名顧客？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT c.`name`, (
    SELECT SUM(d.`price` * o.`qty`)
    FROM `orders` o
    JOIN `drinks` d ON o.`drinks_id` = d.`id`
    WHERE o.`customers_id` = c.`id`
    ) AS `SUM`
FROM `customers` c
ORDER BY `SUM` DESC
LIMIT 2;
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,name,SUM
0,Wendy,620
1,Johny,50


In [13]:
# 3.這個月生日的顧客有誰？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT `name`
FROM `customers`
WHERE MONTH(`birthday`) = MONTH(CURRENT_DATE())
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,name


# 問題2
---
1. 我們目前銷售量最高的飲料是哪一種？
2. 我們目前最熱門（購買顧客最多）的飲料是哪一種？
3. 哪三種飲料為我們帶來最高營業額？
4. 哪三種飲料為我們帶來最多淨收入？

In [14]:
# 1. 我們目前銷售量最高的飲料是哪一種？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT d.`name`, SUM(`qty`) AS `Total_sales`
FROM `orders` o
JOIN `drinks` d ON d.`id` = o.`drinks_id`
GROUP BY `drinks_id`
ORDER BY `Total_sales` DESC
LIMIT 1;
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,name,Total_sales
0,四季春茶,11


In [15]:
# 2. 我們目前最熱門（購買顧客最多）的飲料是哪一種？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT d.`name`, o.CNT
FROM `drinks` AS d
JOIN (
  SELECT `drinks_id`, COUNT(`customers_id`) CNT
  FROM `orders`
  GROUP BY `drinks_id`
) AS o ON d.`id` = o.`drinks_id`
ORDER BY o.`CNT` DESC
LIMIT 1;
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,name,CNT
0,四季春茶,2


In [16]:
# 3. 哪三種飲料為我們帶來最高營業額？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT d.`name`, SUM(d.`price` * o.`qty`) AS Total_sales
FROM `drinks` AS d
JOIN `orders` AS o ON d.`id` = o.`drinks_id`
GROUP BY d.`name`
ORDER BY total_sales DESC
LIMIT 3;
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,name,Total_sales
0,四季春茶,275
1,阿華田,260
2,百香紅茶,135


In [17]:
# 4. 哪三種飲料為我們帶來最多淨收入？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT d.`name`, SUM((d.`price`-d.`cost`) * o.`qty`) AS NET_income
FROM `drinks` AS d
JOIN `orders` AS o ON d.`id` = o.`drinks_id`
GROUP BY d.`name`
ORDER BY NET_income DESC
LIMIT 3;
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,name,NET_income
0,四季春茶,220
1,阿華田,180
2,百香紅茶,105


# 問題3
---
1. 哪種飲料至今沒人買過？
2. 一天中，哪個小時的客人最多？
3. 誰是為我們帶來最多淨收入的兩位顧客？

In [18]:
# 1. 哪種飲料至今沒人買過？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT `name`
FROM `drinks`
WHERE `id` NOT IN (
  SELECT DISTINCT `drinks_id`
  FROM `orders`
);
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,name
0,冰咖啡
1,好喝檸檬水
2,愛玉冰茶


In [19]:
# 2. 一天中，哪個小時的客人最多？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT HOUR(`order_time`) AS hour, COUNT(*) AS guest_count
FROM `orders`
GROUP BY hour
ORDER BY guest_count DESC
LIMIT 1;
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,hour,guest_count
0,14,2


In [20]:
# 3. 誰是為我們帶來最多淨收入的兩位顧客？
conn = conn_login()
cursor = conn.cursor()
cursor.execute('''
SELECT c.`id`, c.`name`, res.`Total NET income`
FROM `customers` AS c
JOIN (
  SELECT o.`customers_id`, SUM((d.`price`-d.`cost`) * o.`qty`) AS 'Total NET income'
  FROM `drinks` AS d
  JOIN `orders` AS o ON o.`drinks_id` = d.`id`
  GROUP BY o.`customers_id`
  LIMIT 2
) AS res ON c.`id` = res.`customers_id`
ORDER BY res.`Total NET income` DESC;
''')
res = cursor.fetchall()
cursor.close()
conn.close()

fetchall_TO_DF(res)

,id,name,Total NET income
0,2,Wendy,465
1,1,Johny,40
